# Day 11: Monkey in the Middle

## Part 1

In [ ]:
from aoc_2023 import core


_example = """Monkey 0:
  Starting items: 79, 98
  Operation: new = old * 19
  Test: divisible by 23
    If true: throw to monkey 2
    If false: throw to monkey 3

Monkey 1:
  Starting items: 54, 65, 75, 74
  Operation: new = old + 6
  Test: divisible by 19
    If true: throw to monkey 2
    If false: throw to monkey 0

Monkey 2:
  Starting items: 79, 60, 97
  Operation: new = old * old
  Test: divisible by 13
    If true: throw to monkey 1
    If false: throw to monkey 3

Monkey 3:
  Starting items: 74
  Operation: new = old + 3
  Test: divisible by 17
    If true: throw to monkey 0
    If false: throw to monkey 1"""
_test = core.read_input("../data/day_11.txt")

In [ ]:
from dataclasses import dataclass
from typing import Callable


@dataclass(frozen=True)
class Monkey:
    index: int
    items: list[str]
    operation: Callable[int, ...]
    divisible_by: int
    on_true: int
    on_false: int

In [ ]:
import math


def parse_monkey(s: str) -> Monkey:
    lines = s.split("\n")
    index = int(lines[0].strip()[len("Monkey"):-1])
    items = [int(item) for item in lines[1].strip()[len("Starting items:"):].split(", ")]
    operation = lines[2].strip()[len("Operation: new = "):]
    divisible_by = int(lines[3].strip()[len("Test:  divisible by"):])
    on_true = int(lines[4].strip()[len("If true: throw to monkey "):])
    on_false = int(lines[5].strip()[len("If false: throw to monkey ")])
    operation = eval(f"lambda old: {operation}")

    return Monkey(
        index=index,
        items=items,
        operation=operation,
        divisible_by=divisible_by,
        on_true=on_true,
        on_false=on_false) 


def parse(s: str) -> list[Monkey]:
    monkeys = [parse_monkey(monkey) for monkey in s.split("\n\n")]
    return monkeys


def calculate(monkey, item, scale):
    return (value := math.trunc(monkey.operation(item) // scale), (value % monkey.divisible_by == 0))


def monkey_business(monkeys: list[Monkey], num_rounds: int, scale: float) -> int:
    # Calculate monkey business :-)
    counters = [0] * len(monkeys)
    
    limit = 1
    for monkey in monkeys:
        limit *= monkey.divisible_by
    
    for round_number in range(num_rounds):
        for i, monkey in enumerate(monkeys):
            counters[i] += len(monkey.items)
            items = [
                # limit prevents from growing uncontrollably
                # while retaining ability to test for divisibility
                # down the road.
                calculate(monkey, int(item) % limit, scale)
                for item in monkey.items
            ]
            monkey.items.clear()
            for value, is_divisible in items:
                if is_divisible:
                    monkeys[monkey.on_true].items.append(value)
                else:
                    monkeys[monkey.on_false].items.append(value)
    first, second, *_ = sorted(counters, reverse=True)
    return first * second

In [ ]:
def part_1(s: str) -> int:
    monkeys = parse(s)
    return monkey_business(monkeys, 20, 3)

In [ ]:
part_1(_example)

10605

In [ ]:
part_1(_test)

182293

## Part 2

In [ ]:
def part_2(s: str) -> int:
    monkeys = parse(s)
    return monkey_business(monkeys, 10000, 1)

In [ ]:
part_2(_example)

2713310158

In [ ]:
part_2(_test)

54832778815